# 0. Download data

1 download the data in huggingface;

[Data](https://huggingface.co/datasets/Jungle15/GDP-HMM_Challenge)

2 change the `npz_path` in the `meta_files/meta_data.csv` depending on the data path on your local machine.  

## 1. Python Environment

The baseline has been tested with Python 3.10, PyTorch 2.1.2, and MONAI 1.4.0. Similar versions should work but have not been tested by organizers. 

The other necessary pakages can be installed by:

pip install -r requirements.txt

## 2. Preprocess Downloaded Data

Preprocess the downloaded data using the code [data_preprocess.py](./data_preprocess.py). Please revise 'csv_root' and 'dataset_save_root' according to the downloaded data path and desired save path for preprocessed data on your local machine.

## 3. Import neccessary packages and Hyperparameters

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from nnunet_mednext import create_mednext_v1
import data_loader_lightning
import yaml

cfig = yaml.load(open('config_files/config_dummy.yaml'), Loader=yaml.FullLoader)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


The config includes two major parts: loader_params and model_params. We will introduce them more in the following. 

In [2]:
cfig['loader_params']

{'train_bs': 1,
 'val_bs': 1,
 'csv_root': 'meta_files/meta_data_dummy.csv',
 'csv_root_validation': 'meta_files/meta_data_sanity_validation.csv',
 'csv_HaN_OAR_priority_root': 'meta_files/HaN_OAR_update.csv',
 'csv_LUNG_OAR_priority_root': 'meta_files/LUNG_OAR_update.csv',
 'scale_dose_dict': 'meta_files/PTV_DICT.json',
 'pat_obj_dict': 'meta_files/Pat_Obj_DICT.json',
 'num_workers': 4,
 'down_HU': -1000,
 'up_HU': 1000,
 'denom_norm_HU': 500,
 'in_size': [96, 128, 160],
 'out_size': [96, 128, 160],
 'CatStructures': False,
 'dose_div_factor': 10}

In [3]:
cfig['model_params']

{'num_input_channels': 6,
 'out_channels': 1,
 'model_id': 'A',
 'kernel_size': 3,
 'deep_supervision': False}

## 3. Data loader

In [4]:
loaders = data_loader_lightning.GetLoader(cfig = cfig['loader_params'])
train_loader =loaders.train_dataloader()
val_loader = loaders.train_val_dataloader()

## 4. Network structure

As mentioned earlier, we use MedNeXt as the backbone. Please follow the MedNeXt official instructions to adjust the structure. The example we use is as below: 

In [5]:
model = create_mednext_v1( num_input_channels = cfig['model_params']['num_input_channels'],
  num_classes = cfig['model_params']['out_channels'],
  model_id = cfig['model_params']['model_id'],          # S, B, M and L are valid model ids
  kernel_size = cfig['model_params']['kernel_size'],   # 3x3x3 and 5x5x5 were tested in publication
  deep_supervision = cfig['model_params']['deep_supervision']   
).to(device)

## 5. Define loss function and optimizer

In [6]:
from Loss import L1_DVH_Loss
optimizer = optim.Adam(model.parameters(), lr=cfig['lr'])
criterion = L1_DVH_Loss

## 6. Training 

Then, you are ready to with training loops.

In [7]:
for epoch in range(cfig['num_epochs']):
    model.train()
    for i, data_dict in enumerate(train_loader):
        # Forward pass
        outputs = model(data_dict['data'].to(device))
        loss = criterion(outputs, data_dict['label'].to(device),data_dict['PTV'].to(device),data_dict['oar_serial'].to(device),data_dict['oar_parallel'].to(device),device)
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print(f"Epoch [{epoch+1}/{cfig['num_epochs']}], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}")


Epoch [1/10], Step [1/4], Loss: 0.8451
Epoch [1/10], Step [2/4], Loss: 0.5782
Epoch [1/10], Step [3/4], Loss: 0.6047
Epoch [1/10], Step [4/4], Loss: 0.4416
Epoch [2/10], Step [1/4], Loss: 0.3344
Epoch [2/10], Step [2/4], Loss: 0.3432
Epoch [2/10], Step [3/4], Loss: 0.3887
Epoch [2/10], Step [4/4], Loss: 0.4147
Epoch [3/10], Step [1/4], Loss: 0.3649
Epoch [3/10], Step [2/4], Loss: 0.3414
Epoch [3/10], Step [3/4], Loss: 0.3452
Epoch [3/10], Step [4/4], Loss: 0.3025
Epoch [4/10], Step [1/4], Loss: 0.3975
Epoch [4/10], Step [2/4], Loss: 0.3536
Epoch [4/10], Step [3/4], Loss: 0.3613
Epoch [4/10], Step [4/4], Loss: 0.2943
Epoch [5/10], Step [1/4], Loss: 0.4379
Epoch [5/10], Step [2/4], Loss: 0.3407
Epoch [5/10], Step [3/4], Loss: 0.3730
Epoch [5/10], Step [4/4], Loss: 0.2729
Epoch [6/10], Step [1/4], Loss: 0.2992
Epoch [6/10], Step [2/4], Loss: 0.2890
Epoch [6/10], Step [3/4], Loss: 0.3367
Epoch [6/10], Step [4/4], Loss: 0.3219
Epoch [7/10], Step [1/4], Loss: 0.2313
Epoch [7/10], Step [2/4],

## 7. Train a complete model.
To train a complete model, please run the codes [train.py](./train.py) with the [configure file](./config_files/config_AGMedNext.yaml) and [train_upkern.py](./train_upkern.py) with the [configure file](./config_files/config_AGMedNext_UpKern.yaml) sequentially. 

Please revise the `npz_path` in the `meta_files/meta_data.csv` depending on the preprocessed data path on your local machine before model training:

python train.py config_files/config_AGMedNext.yaml

python train_upkern.py config_files/config_AGMedNext_UpKern.yaml

python train_upkern.py config_files/config_AGMedNext_UpKern7.yaml

## 8. Model inference
To use the trained model for dose prediction, please run [inference.py](./inference.py)

Example usage:

python inference.py config_files/config_infer.yaml

## 9. Model evaluation
To evaluate the dose prediction accuracy of the trained model, please run [evaluation.py](./evaluation.py)

Example usage:

python evaluation.py